In [79]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [80]:
allFacilities = pd.read_csv("output/describeNetwork.csv", dtype={'primary_naics': object})

In [81]:
# Dioxin reporters from 2013 TRI 
tri2013 = pd.read_csv('data/toxic-release-inventory.ny.2013.geoid.csv')
dioxins_2013 = tri2013[tri2013['chemical']=='DIOXIN AND DIOXIN-LIKE COMPOUNDS']
len(dioxins_2013) # Number of dioxin reporting facilities

20

In [107]:
allChems = pd.unique(tri2013['chemical'].values.ravel())

In [82]:
chem = 'DIOXIN AND DIOXIN-LIKE COMPOUNDS'

In [83]:
allFacilities = allFacilities.drop(['Unnamed: 0', 'Facility','Betweeness','Closeness','Eigenvector'], axis=1)
withChem = allFacilities[allFacilities['Chemicals'].str.contains(chem)]
withoutChem = allFacilities[~allFacilities['Chemicals'].str.contains(chem)]

In [75]:
print 'Num of total facilities:', len(allFacilities)
print 'Num of dioxin-emitting facilities:', len(withDioxins)
print 'Num of non dioxin-emitting facilities:', len(withoutDioxins)

Num of total facilities: 387
Num of dioxin-emitting facilities: 19
Num of non dioxin-emitting facilities: 368


In [74]:
# allFacilities[allFacilities['Chemicals'].str.contains('DIOXIN AND DIOXIN-LIKE COMPOUNDS')]

In [102]:
def chemPickCommunity(c):
    withChemComm = withChem[withDioxins['Community'] == c]
    chemNaics = pd.unique(withChemComm['primary_naics'].values.ravel())
    withoutChemComm = withoutChem[withoutChem['Community'] == c]
#     print chemNaics
    return withoutChemComm[withoutChemComm['primary_naics'].isin(chemNaics)]
#     return withoutChemComm.merge(withChemComm, on='primary_naics', how='inner')

In [106]:
for i in range(6):
#     print len(chemPickCommunity(i))
    print chemPickCommunity(i)

     Degrees  Clustering Coefficient  tri_facility_id primary_naics  \
36       120                0.548179  14302DPNTNBUFFA        325180   
57        91                0.547253  12143BLCRCROUTE        327310   
66        84                0.570281  1461WRCHST681RI        331492   
79        78                0.642691  14150FMCCR37SAW        325180   
96        75                0.650811  14150CRHNT3500R        221112   
159       68                0.730904  14048NGRMH106PO        221112   
232       51                0.697255  14210PVSCH55LEE        325180   
248       41                0.580488  12801MSGLD21ROG        331492   
250       40                0.835897  13209GNRLCWILLI        325180   
259       35                1.000000  11980CTHNS5ZRNB        221112   
260       35                1.000000  12822NDCKC24WHI        221112   
261       35                1.000000  14150CRBRN168CR        325180   
262       35                1.000000  14760NDCKL140MO        221112   
267   